# TAPL Subtyping

## Table of content

NOTE: the title of some sections are modfied to emphasize specific content

- [Chapter 15 Subtyping](#subtyping)
  - [15.1 motivation](#motivation)
  - [15.2 Record](#record)
  - [15.2 S-Arrow](#s-arrow)
  - 15.3 Progress & Preservation (skipped)
  - [15.4 Top & Bottom](#top-bottom)
  - [15.5 Cast, Reference](#cast-ref)
  - 15.6 Coercion Semantics for Subtyping (skipped)
- [Chapter 16 Metatheory of Subtyping](#metatheory)
  - 16.1 Alogrithmic Subtyping
  - 16.2 Alogrithmic Typing
- Chapter 17 An ML Implementation of Subtyping
- [Reference](#reference)

## 15 Subtyping <a name="subtyping"></a>

### 15.1 Motivation <a name="motivation"></a>

$$(\lambda \text{r:{x:Nat}. r.x) {x=0}}$$

$$(\lambda \text{r:{x:Nat}. r.x) {x=0,y=1}}$$

$$\frac{\Gamma \vdash t_1 : T_{11} \rightarrow T_{12} \quad \Gamma \vdash t_2 : T_{11}}{\Gamma \vdash t_1 t_2 : T_{12}} \quad\quad \textbf{(T-APP)}$$

Intuitively, passing {x=0} and {x=0,y=1} to ($\lambda$ r:{x:Nat}. r.x) should both yield 0, but the second application is not typable under our current typing rule. So we extend our rule to allow this kind of operation, where a more informative type ({x:Nat, y:Nat}) is used in the place of a less informative one ({x:Nat}).

Foramally, S is a *subtype* of T written S <: T, mean any term of type S can safely be used in a context where a term of type T is expected

$$\frac{\Gamma \vdash t : S \quad S <: T}{\Gamma \vdash t : T} \quad\quad \textbf{(T-SUB)}$$

Intuitively, our *subtype* relation also have reflexivity and transitivity

$$ S <: S \quad \textbf{(S-REFL)}$$

$$\frac{S <:U \quad U <: T}{S <: T} \quad\quad \textbf{(S-TRANS)}$$

### 15.2 Record <a name="record"></a>

Now let's formalize {x=0, y=1} <: {x=0} as **width subtyping** rule

$$\{l_i:T_i^{i \in 1..n+k}\} <: \{l_i:T_i^{i \in 1..n}\} \quad \textbf{(S-RCDWIDTH)}$$

It might looks counter intuitive that the "smaller" type (subtype) actually has more fields. That is because more fields in the record type means more restriction and a smaller set it can cover, the following [euler diargam](https://en.wikipedia.org/wiki/Euler_diagram) shows this relation, {x:Nat, y:Nat, z:Nat} <: {x:Nat, y:Nat} <: {x:Nat} and {x:Nat, y:Nat} <: {y:Nat}

![record-width](record-width.png)

In S-RcdWidth, we require each field has identitical type, however, this restriction can be loosen, they can have subtype relation, this gives us the **depth subtyping** rule

$$\frac{\text{for each } i \quad S_i <: T_i}{\{l_i:S_i^{i \in 1..n}\} <: \{l_i:T_i^{i \in 1..n}\}}$$

We can combine S-RcdWidth and S-RcdDepth together

$$\frac{
\text{\{ x: Nat, y: Nat, z: Nat\}} <: \text{\{x: Nat, y: Nat\}} \quad 
\text{\{radius: Nat\}} <: \text{\{radius: Nat\}}
}
{\text{\{center: {x: Nat, y: Nat, z: Nat}, radius: Nat\}} <: \text{\{center: {x: Nat, y: Nat}, radius: Nat\}} }$$

However, if we add mutable record, things might break after application

````ocaml
(* NOTE: this example is modifed from [2] for demonstration, it is not valid OCaml nor lambda-calculus *)
let sphere = {center={x=1, y=2, z=3}, radius=2} in
    let circle = sphere in 
        (* move a circle to the center of the axis *)
        (lambda c:{center: {x: Nat, y:Nat}, radius: Nat}. c.center={x=0, y=0}) circle
    (* what happens now? *)
    sphere.center.z
````

In fact, record subtyping is not widely adopted, OCaml is using row-variable polymorphism for records [1].

### 15.2 S-Arrow <a name="s-arrow"></a>

In a high order language, function can be argument to other function, thus we have the following subtyping rule.

$$\frac{T_1 <: S_1 \quad S_2 <: T_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW})$$

Note on the left side of $\rightarrow$, the relation is *contravariant* while on the right side, the relation is *covariant*, which is pretty counter intuitive. One might expect the left side (arugment) is convariant. This confusion arises from two parts:

- in previous example of function application, we replace argument with its subtype. i.e. replace {x:Nat} with {x:Nat, y:Nat}
- only think about the argument of the function, NOT taking function as a whole. [5]

Let's look at a concrete example first, recall the typing rule for subtyping (T-SUB) is *any term of type S can safely be used in a context where a term of type T is expected*, we write a function that accept function as argument. For better illustration, we use a more Java like notation with function name and multiple arguments instead of lambda-calculus style. 

````js
/* move from the start point 10 times using the mover function */
/* ({x:Nat, y:Nat} -> {x:Nat, y:Nat}) -> {x:Nat, y:Nat} -> {x:Nat, y:Nat} */
function start_game(mover:{x:Nat, y:Nat} -> {x:Nat, y:Nat}, p:{x:Nat, y:Nat}) : {x:Nat, y:Nat} {
    for i := 0; i < 10; i++ {
        p := mover(p)
    }
    return p
}

/* walk all the way north until u meet the wall, you are type safe, John Snow */
/* case 0: exact same type */
function walk_north(p:{x:Nat, y:Nat}) : {x:Nat, y:Nat} {
    return {x=p.x, y=p.y-1}
}

/* grab a dragon and fly north */
/* case 1: covariant in both argument and return value, {x:Nat, y:Nat, z:Nat} <: {x:Nat, y:Nat} */
function fly_north(p:{x:Nat, y:Nat, z:Nat}) : {x:Nat, y:Nat, z:Nat} {
    return {x=p.x, y=p.y-1, z=p.z}
}

// TODO: this seems to be useless
/* grab a dragon, but take off and land every time */
/* exact same type */
function fly_north_land(p:{x:Nat, y:Nat}) : {x:Nat, y:Nat} {
    s = {x=p.x, y=p.y-1, z=10} // take off
    return {x=s.x, y=s.y} // land
}

/* case 2: contravariant in both argument and return value */
function walk_north_only(p:{y:Nat}) : {y:Nat} {
    return {y=p.y-1}
}
````

In [19]:
def start_game(mover, p):
    for i in range(10):
        p = mover(p)
    print('{} to x: {} y: {}'.format(mover.__name__, p['x'], p['y']))

def walk_north(p):
#     p['x'] = p['x']
#     p['y'] = p['y'] + 1
    return {'x':p['x'], 'y':p['y'] + 1}

start_game(mover=walk_north, p={'x':0, 'y':0})

walk_north to x: 0 y: 10


In [5]:
def fly_north(p):
    p['x'] = p['x']
    p['y'] = p['y'] + 1
    p['z'] = p['z']
    return p

In [15]:
try:
    start_game(mover=fly_north, p={'x':0, 'y':0})
except KeyError as err:
    print('{} is expected but not presented in mover: {}'.format(err, fly_north.__name__))

'z' is expected but not presented in mover: fly_north


Actually `fly_north` does work sometime, but we want our type system to be sound, not *it might work*

In [16]:
start_game(mover=fly_north, p={'x':0, 'y':0, 'z': 0})

end up in x: 0 y: 10


In [ ]:
def walk_north_only(p):
    return 

## Reference <a name="reference"></a>

Books

- [1] Types and Programming Language (2nd Edition)

Courses

- [2] [Coursera Programming Languages Part C Week 3](https://www.coursera.org/learn/programming-languages-part-c/lecture/mdwdY/subtyping-from-the-beginning)
- [3] [EPFL Type Systems 2004](http://lampwww.epfl.ch/teaching/archive/type_systems/2004/)
- [4] [EPFL Foundations of Software 2017](https://fos2017.github.io/)

Meetup

- [5] [Computational Club (UK) TAPL reading meetup](https://github.com/computationclub/computationclub.github.io/wiki/Types-and-Programming-Languages-Chapter-15-Subtyping-%E2%80%93-Part-1)